# Installation

In [12]:
!pip install lightgbm
!pip install eli5
!pip install shap
!pip install sklearn
!pip install pandas
!pip install numpy
!pip install tqdm

# Imports

In [40]:
# import eli5
# import lightgbm as lgb
import numpy as np
import os
import pandas as pd
# import shap
# import sklearn

# from sklearn.metrics import auc, make_scorer
# from sklearn.model_selection import cross_val_score, GridSearchCV, StratifiedKFold
from tqdm import tqdm

# Data

In [2]:
data_dir = 'data/'
csvs = [x for x in os.listdir(data_dir) if x.endswith('.csv')]
print(csvs)
for csv in csvs:
    print(csv)
    df = pd.read_csv(os.path.join(data_dir, csv))
    print(len(df))
    print(list(df))

['ADMISSIONS.csv', 'CALLOUT.csv', 'CAREGIVERS.csv', 'CPTEVENTS.csv', 'DATETIMEEVENTS.csv', 'DIAGNOSES_ICD.csv', 'DRGCODES.csv', 'D_CPT.csv', 'D_ICD_DIAGNOSES.csv', 'D_ICD_PROCEDURES.csv', 'D_ITEMS.csv', 'D_LABITEMS.csv', 'ICUSTAYS.csv', 'INPUTEVENTS_CV.csv', 'INPUTEVENTS_MV.csv', 'LABEVENTS.csv', 'MICROBIOLOGYEVENTS.csv', 'OUTPUTEVENTS.csv', 'PATIENTS.csv', 'PRESCRIPTIONS.csv', 'PROCEDUREEVENTS_MV.csv', 'PROCEDURES_ICD.csv', 'SERVICES.csv', 'TRANSFERS.csv']
ADMISSIONS.csv
58976
['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'ADMITTIME', 'DISCHTIME', 'DEATHTIME', 'ADMISSION_TYPE', 'ADMISSION_LOCATION', 'DISCHARGE_LOCATION', 'INSURANCE', 'LANGUAGE', 'RELIGION', 'MARITAL_STATUS', 'ETHNICITY', 'EDREGTIME', 'EDOUTTIME', 'DIAGNOSIS', 'HOSPITAL_EXPIRE_FLAG', 'HAS_CHARTEVENTS_DATA']
CALLOUT.csv
34499
['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'SUBMIT_WARDID', 'SUBMIT_CAREUNIT', 'CURR_WARDID', 'CURR_CAREUNIT', 'CALLOUT_WARDID', 'CALLOUT_SERVICE', 'REQUEST_TELE', 'REQUEST_RESP', 'REQUEST_CDIFF', 'REQUEST_MRSA', '

c:\programdata\miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (4,5,7,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


573146
['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'COSTCENTER', 'CHARTDATE', 'CPT_CD', 'CPT_NUMBER', 'CPT_SUFFIX', 'TICKET_ID_SEQ', 'SECTIONHEADER', 'SUBSECTIONHEADER', 'DESCRIPTION']
DATETIMEEVENTS.csv


c:\programdata\miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


4485937
['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'ITEMID', 'CHARTTIME', 'STORETIME', 'CGID', 'VALUE', 'VALUEUOM', 'WARNING', 'ERROR', 'RESULTSTATUS', 'STOPPED']
DIAGNOSES_ICD.csv
651047
['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'SEQ_NUM', 'ICD9_CODE']
DRGCODES.csv
125557
['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'DRG_TYPE', 'DRG_CODE', 'DESCRIPTION', 'DRG_SEVERITY', 'DRG_MORTALITY']
D_CPT.csv
134
['ROW_ID', 'CATEGORY', 'SECTIONRANGE', 'SECTIONHEADER', 'SUBSECTIONRANGE', 'SUBSECTIONHEADER', 'CODESUFFIX', 'MINCODEINSUBSECTION', 'MAXCODEINSUBSECTION']
D_ICD_DIAGNOSES.csv
14567
['ROW_ID', 'ICD9_CODE', 'SHORT_TITLE', 'LONG_TITLE']
D_ICD_PROCEDURES.csv
3882
['ROW_ID', 'ICD9_CODE', 'SHORT_TITLE', 'LONG_TITLE']
D_ITEMS.csv
12487
['ROW_ID', 'ITEMID', 'LABEL', 'ABBREVIATION', 'DBSOURCE', 'LINKSTO', 'CATEGORY', 'UNITNAME', 'PARAM_TYPE', 'CONCEPTID']
D_LABITEMS.csv
753
['ROW_ID', 'ITEMID', 'LABEL', 'FLUID', 'CATEGORY', 'LOINC_CODE']
ICUSTAYS.csv
61532
['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID',

c:\programdata\miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (7,9,17,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


17527935
['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'CHARTTIME', 'ITEMID', 'AMOUNT', 'AMOUNTUOM', 'RATE', 'RATEUOM', 'STORETIME', 'CGID', 'ORDERID', 'LINKORDERID', 'STOPPED', 'NEWBOTTLE', 'ORIGINALAMOUNT', 'ORIGINALAMOUNTUOM', 'ORIGINALROUTE', 'ORIGINALRATE', 'ORIGINALRATEUOM', 'ORIGINALSITE']
INPUTEVENTS_MV.csv
3618991
['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'STARTTIME', 'ENDTIME', 'ITEMID', 'AMOUNT', 'AMOUNTUOM', 'RATE', 'RATEUOM', 'STORETIME', 'CGID', 'ORDERID', 'LINKORDERID', 'ORDERCATEGORYNAME', 'SECONDARYORDERCATEGORYNAME', 'ORDERCOMPONENTTYPEDESCRIPTION', 'ORDERCATEGORYDESCRIPTION', 'PATIENTWEIGHT', 'TOTALAMOUNT', 'TOTALAMOUNTUOM', 'ISOPENBAG', 'CONTINUEINNEXTDEPT', 'CANCELREASON', 'STATUSDESCRIPTION', 'COMMENTS_EDITEDBY', 'COMMENTS_CANCELEDBY', 'COMMENTS_DATE', 'ORIGINALAMOUNT', 'ORIGINALRATE']
LABEVENTS.csv
27854055
['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'ITEMID', 'CHARTTIME', 'VALUE', 'VALUENUM', 'VALUEUOM', 'FLAG']
MICROBIOLOGYEVENTS.csv
631726
['ROW_ID', 'SUBJ

c:\programdata\miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


4156450
['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'STARTDATE', 'ENDDATE', 'DRUG_TYPE', 'DRUG', 'DRUG_NAME_POE', 'DRUG_NAME_GENERIC', 'FORMULARY_DRUG_CD', 'GSN', 'NDC', 'PROD_STRENGTH', 'DOSE_VAL_RX', 'DOSE_UNIT_RX', 'FORM_VAL_DISP', 'FORM_UNIT_DISP', 'ROUTE']
PROCEDUREEVENTS_MV.csv
258066
['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'STARTTIME', 'ENDTIME', 'ITEMID', 'VALUE', 'VALUEUOM', 'LOCATION', 'LOCATIONCATEGORY', 'STORETIME', 'CGID', 'ORDERID', 'LINKORDERID', 'ORDERCATEGORYNAME', 'SECONDARYORDERCATEGORYNAME', 'ORDERCATEGORYDESCRIPTION', 'ISOPENBAG', 'CONTINUEINNEXTDEPT', 'CANCELREASON', 'STATUSDESCRIPTION', 'COMMENTS_EDITEDBY', 'COMMENTS_CANCELEDBY', 'COMMENTS_DATE']
PROCEDURES_ICD.csv
240095
['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'SEQ_NUM', 'ICD9_CODE']
SERVICES.csv
73343
['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'TRANSFERTIME', 'PREV_SERVICE', 'CURR_SERVICE']
TRANSFERS.csv
261897
['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'DBSOURCE', 'EVENTTYPE', 'PREV_CAREUNIT', 'CUR

In [18]:
for name in ['ADMISSIONS.csv', 'PATIENTS.csv', 'ICUSTAYS.csv']:
    print(name)
    df = pd.read_csv(os.path.join(data_dir, name))
    print(len(df))
    # print(np.count_nonzero(~np.isnan(df['DRG_MORTALITY'].values)) / len(df['DRG_MORTALITY'].values))
    try:
        print('SUBJECT_ID: ' + str(len(df['SUBJECT_ID'].values)))
        print('Unique SUBJECT_ID: ' + str(len(np.unique(df['SUBJECT_ID'].values))))
        print('HADM_ID: ' + str(len(df['HADM_ID'].values)))
        print('Unique HADM_ID: ' + str(len(np.unique(df['HADM_ID'].values))))
    except:
        pass
    print()

ADMISSIONS.csv
58976
SUBJECT_ID: 58976
Unique SUBJECT_ID: 46520
HADM_ID: 58976
Unique HADM_ID: 58976

PATIENTS.csv
46520
SUBJECT_ID: 46520
Unique SUBJECT_ID: 46520

ICUSTAYS.csv
61532
SUBJECT_ID: 61532
Unique SUBJECT_ID: 46476
HADM_ID: 61532
Unique HADM_ID: 57786



In [70]:
joint_df = pd.read_csv(os.path.join(data_dir, 'ADMISSIONS.csv'))
subject_ids = df['SUBJECT_ID'].values
hadm_ids = df['HADM_ID'].values

In [71]:
joint_df

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
0,21,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2196-04-09 10:06:00,2196-04-09 13:24:00,BENZODIAZEPINE OVERDOSE,0,1
1,22,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,WHITE,NaN,NaN,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1
2,23,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,NaN,NaN,BRAIN MASS,0,1
3,24,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,NaN,PROTESTANT QUAKER,SINGLE,WHITE,NaN,NaN,INTERIOR MYOCARDIAL INFARCTION,0,1
4,25,25,129635,2160-11-02 02:06:00,2160-11-05 14:55:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2160-11-02 01:01:00,2160-11-02 04:27:00,ACUTE CORONARY SYNDROME,0,1
5,26,26,197661,2126-05-06 15:16:00,2126-05-13 15:00:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Medicare,NaN,CATHOLIC,SINGLE,UNKNOWN/NOT SPECIFIED,NaN,NaN,V-TACH,0,1
6,27,27,134931,2191-11-30 22:16:00,2191-12-03 14:45:00,NaN,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Private,NaN,CATHOLIC,NaN,WHITE,NaN,NaN,NEWBORN,0,1
7,28,28,162569,2177-09-01 07:15:00,2177-09-06 16:00:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,WHITE,NaN,NaN,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1
8,29,30,104557,2172-10-14 14:17:00,2172-10-19 14:37:00,NaN,URGENT,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,UNKNOWN/NOT SPECIFIED,NaN,NaN,UNSTABLE ANGINA\CATH,0,1
9,30,31,128652,2108-08-22 23:27:00,2108-08-30 15:00:00,2108-08-30 15:00:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,Medicare,NaN,CATHOLIC,MARRIED,WHITE,NaN,NaN,STATUS EPILEPTICUS,1,1


In [72]:
# Add a column to the dataframe based on SUBJECT_ID.
patients_df = pd.read_csv(os.path.join(data_dir, 'PATIENTS.csv'))
gender_arr = []
for subject_id in subject_ids:
    gender_arr.append(patients_df.loc[patients_df['SUBJECT_ID'] == subject_id]['GENDER'].values[0])
print(list(zip(subject_ids, gender_arr))[:5])
print(patients_df.loc[patients_df['SUBJECT_ID'].isin(subject_ids[:5])])
joint_df = joint_df.assign(GENDER=gender_arr)
joint_df

[(22, 'F'), (23, 'M'), (23, 'M'), (24, 'M'), (25, 'M')]
     ROW_ID  SUBJECT_ID GENDER                  DOB  DOD DOD_HOSP DOD_SSN  \
240      19          22      F  2131-05-07 00:00:00  NaN      NaN     NaN   
241      20          23      M  2082-07-17 00:00:00  NaN      NaN     NaN   
242      21          24      M  2100-05-31 00:00:00  NaN      NaN     NaN   
243      22          25      M  2101-11-21 00:00:00  NaN      NaN     NaN   

     EXPIRE_FLAG  
240            0  
241            0  
242            0  
243            0  


,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA,GENDER
0,21,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2196-04-09 10:06:00,2196-04-09 13:24:00,BENZODIAZEPINE OVERDOSE,0,1,F
1,22,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,WHITE,NaN,NaN,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1,M
2,23,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,NaN,NaN,BRAIN MASS,0,1,M
3,24,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,NaN,PROTESTANT QUAKER,SINGLE,WHITE,NaN,NaN,INTERIOR MYOCARDIAL INFARCTION,0,1,M
4,25,25,129635,2160-11-02 02:06:00,2160-11-05 14:55:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2160-11-02 01:01:00,2160-11-02 04:27:00,ACUTE CORONARY SYNDROME,0,1,M
5,26,26,197661,2126-05-06 15:16:00,2126-05-13 15:00:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Medicare,NaN,CATHOLIC,SINGLE,UNKNOWN/NOT SPECIFIED,NaN,NaN,V-TACH,0,1,M
6,27,27,134931,2191-11-30 22:16:00,2191-12-03 14:45:00,NaN,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Private,NaN,CATHOLIC,NaN,WHITE,NaN,NaN,NEWBORN,0,1,F
7,28,28,162569,2177-09-01 07:15:00,2177-09-06 16:00:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,WHITE,NaN,NaN,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1,M
8,29,30,104557,2172-10-14 14:17:00,2172-10-19 14:37:00,NaN,URGENT,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,UNKNOWN/NOT SPECIFIED,NaN,NaN,UNSTABLE ANGINA\CATH,0,1,M
9,30,31,128652,2108-08-22 23:27:00,2108-08-30 15:00:00,2108-08-30 15:00:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,Medicare,NaN,CATHOLIC,MARRIED,WHITE,NaN,NaN,STATUS EPILEPTICUS,1,1,M


In [ ]:
# Add a column to the dataframe based on SUBJECT_ID.
dob_arr = []
for subject_id in subject_ids:
    dob_arr.append(patients_df.loc[patients_df['SUBJECT_ID'] == subject_id]['DOB'].values[0])
print(list(zip(subject_ids, dob_arr))[:5])
print(patients_df.loc[patients_df['SUBJECT_ID'].isin(subject_ids[:5])])
joint_df = joint_df.assign(DOB=dob_arr)
joint_df

In [ ]:
# Add a column to the dataframe based on HADM_ID.
icu_df = pd.read_csv(os.path.join(data_dir, 'ICUSTAYS.csv'))
los_arr = []
for hadm_id in hadm_ids:
    los = icu_df.loc[icu_df['HADM_ID'] == hadm_id]['LOS'].values
    if len(los) == 1:
        los_arr.append(los[0])
    else:
        los_arr.append(np.NaN)
print(list(zip(hadm_ids, los_arr))[:5])
print(icu_df.loc[icu_df['HADM_ID'].isin(hadm_ids[:5])])
joint_df = joint_df.assign(LOS=los_arr)

# Filter rows that are NaN in the new column.
joint_df = joint_df.dropna(subset=['LOS'])

joint_df

# Modeling

In [5]:
# class BaseModel(object):
#     """Abstract base model class."""

#     def __init__(self, k_folds, tuning_metric, num_trials, confidence_level):
#         """Initialize parameters."""
#         self.k_folds = k_folds
#         self.tuning_metric = tuning_metric
#         self.confidence_level = confidence_level
#         self.num_trials = num_trials

#         self.eval_fn = sklearn.metrics.auc
#         self.model = None
#         self.model_fn = None
#         self.best_params = None
#         self.fixed_parameters = {}
#         self.tuning_parameters = {}

#     def train(self, x, y, feature_names=[]):
#         """Train a model on the inputs and labels provided.

#         Args:
#             x (numpy.ndarray): Input data to train on
#             y (numpy.ndarray): Labels corresponding to the input data
#         """

#         print(f'Input shape is {x.shape}')

#         initial_parameters = self.fixed_parameters
#         initial_parameters.update(
#             {param: val[0]
#                 for param, val in self.tuning_parameters[0].items()})

#         # Handle single output model instantiation
#         self.model = self.model_fn(**initial_parameters)

#         scoring_fn = make_scorer(
#             self.eval_fn,
#             greater_is_better=True)

#         best_score = float('-inf')
#         best_model = self.model
#         folds = list(StratifiedKFold(self.k_folds).split(x, y))

#         for i in tqdm(range(len(self.tuning_parameters))):
#             if logger is not None:
#                 logger.log(f"Tuning step {i+1}:", self.tuning_parameters[i])

#             # Perform a grid search over the parameters specified
#             # in self.tuning_parameters[i].
#             # Set verbose to 2 for printing each step of the grid search.
#             # Set n_jobs to -1 to use all processors.
#             grid_searcher = GridSearchCV(estimator=self.model,
#                  param_grid=self.tuning_parameters[i],
#                  scoring=scoring_fn,
#                  iid=False,
#                  cv=folds,
#                  refit=False,
#                  n_jobs=-1)
#             grid_searcher.fit(x, y)

#             # Record the entire CSV regardless for future reference
#             results = grid_searcher.cv_results_
#             df = pd.DataFrame(results)

#             if grid_searcher.best_score_ > best_score:

#                 best_params = grid_searcher.best_params_
#                 best_score = grid_searcher.best_score_

#                 fixed_parameters = self.fixed_parameters
#                 fixed_parameters.update({param: val
#                                          for param, val in best_params.items()})
#                 best_model = self.model_fn(**fixed_parameters)


#         self.model = best_model

#         # K-fold cross validation.
#         sample_ids = np.zeros(x.shape[0])
#         ground_truth = np.zeros(x.shape[0])
#         predictions = np.zeros(x.shape[0])
#         shap_values = np.zeros(x.shape)
#         shap_interaction_values = np.zeros(shape=(x.shape[0], x.shape[1], x.shape[1]))
#         expected_values = []
#         for train_ids, test_ids in folds:
#             x_train = x[train_ids]
#             y_train = y[train_ids]
#             self.model.fit(x_train, y_train)

#             explanation = eli5.explain_weights_lightgbm(
#                 self.model,
#                 feature_names=feature_names,
#                 top=25)
#             logger.log(eli5.format_as_text(explanation))

#             x_test = x[test_ids]
#             y_test = y[test_ids]
#             y_pred = self.model.predict(x_test)

#             sample_ids[test_ids] = test_ids
#             ground_truth[test_ids] = y_test
#             predictions[test_ids] = y_pred

#             explainer = shap.TreeExplainer(self.model)
#             shap_values[test_ids] = explainer.shap_values(x_test)
#             shap_interaction_values[test_ids] = explainer.shap_interaction_values(x_test)
#             expected_values.append(explainer.expected_value)

#         shap_dir = logger.outputs_dir / 'shap'
#         shap_dir.mkdir()

#         expected_value = np.mean(expected_values)
#         logger.log_output(expected_values, 'shap/expected_values')
#         logger.log_output(shap_values, 'shap/shap_values')
#         logger.log_output(shap_interaction_values,
#                           'shap/shap_interaction_values')
#         logger.log_output(x, 'shap/x')
#         logger.log_output(feature_names, 'shap/feature_names')

#         record = np.hstack((sample_ids.reshape(-1, 1),
#                             ground_truth.reshape(-1, 1),
#                             predictions.reshape(-1, 1)))
#         print(f'Record shape is {record.shape}')

#         # Bootstrapping to obtain confidence intervals.
#         scores = []
#         num_successes = 0
#         num_tries = 0
#         indices = list(range(len(ground_truth)))
#         while (num_successes < self.num_trials):
#             # Limit the number of tries.
#             num_tries += 1
#             if num_tries > 2 * self.num_trials:
#                 raise ValueError(
#                     "Too many unsuccessful tries to compute metric.")

#             # Handle case where only one class is included by indices.
#             try:
#                 new_indices = np.random.choice(indices, size=len(indices))
#                 score = EVAL_FNS['C-STATS'](
#                     ground_truth[new_indices], predictions[new_indices],
#                     throw_error=True)
#                 scores.append(score)
#                 num_successes += 1
#             except:
#                 pass

#         mean = np.mean(scores)
#         scores.sort()
#         # Computed using the basic bootstrap
#         lower = 2 * mean - scores[
#             int(((1 + self.confidence_level) / 2) * num_successes)]
#         upper = 2 * mean - scores[
#             int(((1 - self.confidence_level) / 2) * num_successes)]
#         logger.log(f'Lower bound: {lower}')
#         logger.log(f'Mean: {mean}')
#         logger.log(f'Upper bound: {upper}')

#         cs = np.mean(cross_val_score(
#             self.model, x, y, cv=folds, scoring=scoring_fn))
#         logger.log(f'CV {self.tuning_metric}: {cs}')

#     def get_folds(self, x, y):
#         # TODO(ndass): try to clean this up
#         values, counts = np.unique(y, return_counts=True)
#         below_k = [i for i in np.argsort(counts) if counts[i] < self.k_folds]
#         shifts = {}
#         # Merge minority classes until there are more examples than folds.
#         while below_k:
#             i = below_k[0]

#             # Find the closest merge location on the left.
#             pos = -1
#             while (i + pos) >= 0 and counts[i + pos] == 9999999999999:
#                 pos -= 1
#             left = (
#                 counts[i + pos] if (i + pos) >= 0 else float("inf"), i + pos)

#             # Find the closest merge location on the right.
#             pos = 1
#             while (i + pos) < len(values) and counts[i + pos] == 9999999999999:
#                 pos += 1
#             right = (
#                 counts[i + pos] if (i + pos) < len(values) else float("inf"),
#                 i + pos)

#             # Merge the current minority class with the class on the left or
#             # right that has the fewest occurences.
#             shift = min(left, right)[1]
#             counts[shift] += counts[i]
#             counts[i] = 9999999999999
#             if values[i] in shifts.values():
#                 for key in shifts:
#                     if shifts[key] == values[i]:
#                         shifts[key] = values[shift]
#             shifts[values[i]] = values[shift]

#             # Recompute the classes that occur less than fold times.
#             below_k = [
#                 i for i in np.argsort(counts) if counts[i] < self.k_folds]

#         # Update the minority classes of y.
#         values = list(values)
#         y_temp = np.copy(y)
#         for i in range(len(y_temp)):
#             if counts[values.index(y_temp[i])] == 9999999999999:
#                 y_temp[i] = shifts[y_temp[i]]
#         _, counts = np.unique(y_temp, return_counts=True)
#         assert not [i for i in np.argsort(counts) if counts[i] < self.k_folds]

#         # Run StratifiedKFold on the modified version of y to get proper
#         # indices.
#         return list(StratifiedKFold(self.k_folds).split(x, y_temp))

#     def refit_on_full(self, train_x, train_y):
#         """Refit model on full data.

#         This is to avoid using refit=True flag on GridSearchCV.
#         Before testing, we refit the best model on train+dev.

#         Arguments:
#             train_x: Input data to predict labels for
#             train_y: Input labels
#         """
#         self.model.fit(train_x, train_y)

#     def predict(self, x):
#         """Predict labels for the inputs.

#         Args:
#             x (list[list[int]]): Input data to predict labels for

#         Returns:
#             (list[int]): The predicted labels for the input data
#         """
#         raise NotImplementedError('Subclass of BaseModel must implement '
#                                   'predict.')

# Modeling

In [6]:
# class SklearnModel(BaseModel):
#     """Abstract sklearn model class."""

#     def predict(self, x):
#         """See BaseModel.predict."""
#         return self.model.predict(x)

In [7]:
# class LGBMRegressor(SklearnModel):
#     """LightGBM regressor."""

#     def __init__(self, k_folds, tuning_metric, num_trials, confidence_level,
#                  **kwargs):
#         super(LGBMRegressor, self).__init__(
#             k_folds=k_folds, tuning_metric=tuning_metric,
#             num_trials=num_trials, confidence_level=confidence_level)

#         self.model_fn = lgb.LGBMRegressor

#         # Set the hyperparameters to sweep with grid search.
#         self.tuning_parameters = [{
#             'min_child_samples': [0, 10, 20],
#             # 'class_weight':['balanced'],
#             # 'max_bin': [4, 64, 128],
#             'max_depth': [2, 4, 8],
#             'num_leaves': [3, 7, 15],
#             # 'min_split_gain': [0, 0.1],
#             'min_child_weight': [0],
#             # 'reg_alpha': [0.],
#             # 'reg_lambda': [0., 0.01],
#             'n_estimators': [5, 10, 100],
#         }]

In [8]:
# class LGBMClassifier(SklearnModel):
#     """LightGBM classifier."""

#     def __init__(self, k_folds, tuning_metric, num_trials, confidence_level,
#                  **kwargs):
#         super(LGBMClassifier, self).__init__(
#             k_folds=k_folds, tuning_metric=tuning_metric,
#             num_trials=num_trials, confidence_level=confidence_level)

#         self.model_fn = lgb.LGBMClassifier

#         # Set the hyperparameters to sweep with grid search.
#         self.tuning_parameters = [{
#             'class_weight':['balanced'],
#             #'learning_rate': [0.01, 0.1],
#             # 'num_leaves': list(7 * 2**exp for exp in range(2, 7)),
#             #'min_data_in_leaf': [25, 50, 100, 150],
#             # 'max_depth': [1, 2, 3, 4, 5, 10],
#             # 'n_estimators': [3, 5, 10, 30]
#         }]